<a href="https://colab.research.google.com/github/Otter333/nohehe/blob/main/hw02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW02 Web Crawler

## 題目

使用網路爬蟲與 Gradio 建立互動介面，查詢高雄市捷運車站資訊並顯示地圖位置。

## 摘要

本報告透過 Python 網路爬蟲技術，自高雄市政府資料開放平台取得捷運車站的基本資訊，並儲存為結構化的 CSV 格式。進一步整合 Gradio 套件，建立可互動的選單式使用者介面，輸入車站編號即可查詢其中英文名稱與地圖位置。此系統可作為旅遊、通勤或教育用途之輔助工具。

## 引言

隨著智慧城市與開放資料的興起，政府機關釋出大量有用的公共資訊。高雄市政府提供之捷運車站資料包含各站的編號、名稱與經緯度等資訊，若能透過程式自動擷取並視覺化展示，將大幅提升查詢效率與便利性。

本次專案目標是：

  - 利用網路爬蟲自動取得 JSON 格式的車站資料；

  - 儲存為 CSV 以利分析；

  - 使用 Gradio 製作互動式查詢系統，查詢車站資訊並在地圖上顯示位置。


## 方法

- 目標網站描述
  - 網站名稱：高雄市政府資料開放平台

  - 資料集連結：
https://data.kcg.gov.tw/dataset/2629db6d-98ec-4b24-8ab1-5dc34d998460

  - 資料格式：JSON，包含站點編號、中文與英文名稱、經緯度等欄位

________________

- 工具與技術
  - 使用 requests 下載 JSON 資料

  - 使用 csv 模組儲存為 CSV 檔

  - 使用 Gradio 建立互動介面

  - 使用 OpenStreetMap 顯示地理位置

- 從高雄市政府網站下載 JSON 資料

  - 說明：這段程式會將高雄市政府資料平台提供的捷運車站 JSON 檔案下載下來，轉換為 Python 資料格式（list of dictionaries），作為後續儲存與處理的基礎。


In [ ]:
import requests

# 資料來源 URL（高雄市捷運站點資料，JSON 格式）
url = 'https://data.kcg.gov.tw/dataset/2629db6d-98ec-4b24-8ab1-5dc34d998460/resource/662f822c-632b-4cb3-80f6-b5ac053d210b/download/11311.json'

# 發送 GET 請求下載資料
response = requests.get(url)

# 確認回應狀態碼為 200 表示成功
response.status_code

# 將 JSON 轉為 Python list 儲存
stats = response.json()

# 檢查第一筆資料內容
stats[0]


- 將 JSON 資料儲存為 CSV 檔案

  - 說明：這段將剛剛下載的 JSON 資料，依照每一欄的欄位名稱與資料內容，儲存為一個 stats.csv 檔案，方便後續查詢與處理。


In [ ]:
import csv

# 建立要寫入的資料列表，第一列為欄位名稱
wtr = []
wtr.append(list(stats[0].keys()))  # 加入欄位名稱
for it in stats:
    wtr.append(list(it.values()))  # 每筆資料轉為 list 加入

# 將資料寫入 stats.csv
with open('stats.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(wtr)

- 重新讀取 CSV 檔案以供查詢使用

  - 說明：這段程式把 stats.csv 重新讀回記憶體，儲存在 stats 這個變數中。之後會用這份資料製作互動介面。

In [ ]:
import csv

# 讀取剛儲存好的 CSV 檔
with open("stats.csv", "r") as file:
    reader = csv.reader(file)
    stats = list(reader)  # 將 CSV 資料讀成 list，stats[0] 是標題列

- 安裝 Gradio 套件

  - 說明：安裝 Gradio 套件，以便建立 Web 互動式使用者介面（UI）。這行在 Colab 環境中執行即可安裝。



In [ ]:
!pip install gradio -q

- 定義查詢功能與地圖顯示函數

  - 說明：

    stat_info()：根據使用者選擇的車站 ID，回傳車站編號、中文名稱、英文名稱與地圖 HTML。

    get_html()：依據經緯度產生 OpenStreetMap 地圖 iframe，用於嵌入頁面。



In [ ]:
import gradio as gr

# 查詢指定車站 ID 的資訊與地圖
def stat_info(id):
    info = ''
    for it in stats[1:]:  # stats[0] 是欄位名稱，跳過
        if it[1] == id:  # it[1] 是車站編號
            info = f'車站編號: {it[1]}\n中文名稱: {it[2]}\n英文名稱: {it[3]}'
            html = get_html(it[4], it[5])  # it[4] = 緯度, it[5] = 經度
    return info, html

# 根據經緯度產生 OpenStreetMap 地圖 iframe HTML
def get_html(lat, lon):
    lat = float(lat)
    lon = float(lon)
    html = f'''
    <div style="width: 100%; height: 400px;">
        <iframe
            width="100%"
            height="100%"
            frameborder="0"
            scrolling="no"
            marginheight="0"
            marginwidth="0"
            src="https://www.openstreetmap.org/export/embed.html?bbox={lon-0.01}%2C{lat-0.01}%2C{lon+0.01}%2C{lat+0.01}&layer=mapnik&marker={lat}%2C{lon}">
        </iframe>
    </div>
    '''
    return html

- 建立 Gradio 互動式選單介面

  - 說明：
  這是本專案的核心介面，使用 Gradio 提供簡單的下拉選單，使用者選擇車站編號後會顯示：

    車站資訊（編號、中英文名稱）對應位置的地圖

In [ ]:
# 準備所有車站編號作為選單選項
menu = []
for it in stats[1:]:
    menu.append(it[1])  # it[1] 為車站 ID

# 建立 Gradio 介面
gr.Interface(
    fn=stat_info,  # 查詢函數
    inputs=gr.Dropdown(choices=menu, label="車站編號: "),  # 使用 dropdown 讓使用者選擇車站
    outputs=[gr.Textbox(label="車站資訊"), gr.HTML(label="車站地圖")]  # 顯示文字與地圖
).launch()

## 結果


- 資料存儲方式

  - 所有車站資料已儲存為 stats.csv，內容包含：


| 車站編號   | 中文名稱 | 英文名稱              | 緯度      | 經度       |
| ---------- | -------- | --------------------- | --------- | ---------- |
| R3         | 小港     | Siaogang              | 22.5653   | 120.3377   |
| R10/A3     | 美麗島   | Formosa Boulevard     | 22.6306   | 120.3014   |
| ...        | ...      | ...                   | ...       | ...        |


- 查詢介面

  - 可透過下拉選單選擇站點

  - 顯示站名與 OpenStreetMap 標記地圖

In [7]:
# 匯入模組
import json, csv
import gradio as gr

# 讀取本地 JSON 檔
with open('/content/11311.json', 'r', encoding='utf-8') as f:
    stats_json = json.load(f)

# 寫入 CSV
wtr = [list(stats_json[0].keys())]
for it in stats_json:
    wtr.append(list(it.values()))

with open('stats.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(wtr)

# 重新讀取 CSV 為 list
with open('stats.csv', 'r') as file:
    reader = csv.reader(file)
    stats = list(reader)

# 定義地圖 HTML
def get_html(lat, lon):
    lat = float(lat)
    lon = float(lon)
    html = f'''
    <div style="width: 100%; height: 400px;">
        <iframe
            width="100%"
            height="100%"
            frameborder="0"
            scrolling="no"
            marginheight="0"
            marginwidth="0"
            src="https://www.openstreetmap.org/export/embed.html?bbox={lon-0.01}%2C{lat-0.01}%2C{lon+0.01}%2C{lat+0.01}&layer=mapnik&marker={lat}%2C{lon}">
        </iframe>
    </div>
    '''
    return html

# 查詢車站資訊
def stat_info(id):
    info = '查無資料'
    html = ''
    for it in stats[1:]:
        if it[1] == id:
            info = f'車站編號: {it[1]}\n中文名稱: {it[2]}\n英文名稱: {it[3]}'
            html = get_html(it[4], it[5])
            break
    return info, html

# 建立介面
menu = [it[1] for it in stats[1:]]
gr.Interface(
    fn=stat_info,
    inputs=gr.Dropdown(choices=menu, label="車站編號:"),
    outputs=[gr.Textbox(label="車站資訊"), gr.HTML(label="車站地圖")]
).launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2c51557eeab7e06c6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 問題與挑戰

- 技術挑戰
  - 原始資料為 JSON 格式，需解析後儲存

  - Gradio 中地圖顯示需嵌入 HTML iframe 並計算地圖範圍

- 資料限制
  - 部分車站缺乏完整地點描述

  - 地圖僅以經緯度標記，無街景或照片補充

## 結論

本專案成功從高雄市政府開放資料平台擷取捷運車站資料，並建立互動式介面供使用者查詢與定位。此系統具備良好的可擴充性，未來可加入路線資訊、車站出口照片等，進一步提升實用價值。亦可應用於其他城市或不同類型的開放資料。

## 參考文獻

高雄市政府資料開放平台：https://data.kcg.gov.tw

Python Requests 文件：https://docs.python-requests.org/

Gradio 官方文件：https://www.gradio.app

OpenStreetMap：https://www.openstreetmap.org

